In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from Data import*

bom_df = pd.read_excel("Bill of Material (mrp.bom)updated.xlsx")
stock_df = pd.read_excel("Reordering Rule.xlsx")

Stockproducts = list(stock_df['Product'])
#Machines_df = pd.read_excel("Maintenance Equipment.xlsx")
#Machines =list(Machines_df['']
Machines = ['M3-01 - Accuwell - 3axis - Manual (FR3_01)','M5-03 - Haas - 5axis - Manual (UMC750)','M3-02 - Accuwell - 3axis - Manual (FR3_02)','M3-03 - Haas - 3axis - Manual (VF2SS)']
AutomatedMachines = ['M5-04 - Haas - 5axis  - Manual (UMC750SS)','M5-01 - Accuwell - 5axis - Pallet automaton (FR5_01)','M4-01 - Accuwell -  4axis - Conveyor automation (FR4_01)','M4-02 - Accuwell - 4axis - Conveyor automation (FR4_02)','M5-02 - Accuwell - 5axis - Conveyor automation (FR5_02)']
Manual = ['MP3 - Material preparation - pre-cutted, only picking (UITG_M)','MP2 - Material preparation - manual sawing (ZGN_02)','BW - Benchwork (BKW_01)','PACK - Packaging (VERP_P)','MP1 - Material preparation - automatic sawing (ZGN_01)']

DataMgr = DataManager()
DataMgr.getResources().clear()
DataMgr.getProducts().clear()

def MakeProduct (dbmgr,product,stock):

    if (product.find("[")>-1) and (product.find("]")>-1):
        productnumber = product[product.find("[")+1:product.find("]")];
        productname = product[product.find("]")+1:];
        if not "["+productnumber+"] "+productname in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),"["+productnumber+"] "+productname,productnumber,stock);
            return newproduct
        else:
            return dbmgr.getProducts()["["+productnumber+"] "+productname]
    else:
        if not product in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),product,product,stock);
            return newproduct
        else:
            return dbmgr.getProducts()[product]

def MakeOperation(dbmgr,operation,prod,resource,duration):
    myresource=None;
    if not resource in dbmgr.getResources():
        if resource in Machines or resource in AutomatedMachines:
            if resource in Machines:
              myresource = Resource(len(dbmgr.getResources()),"Machine",resource,15);
              myresource.setAutomated(False);
              myresource.setOperatingEffort(0.3);
              myresource.setAvailableShift("All");
              dbmgr.getResources()[resource]=myresource;
            else:
               myresource = Resource(len(dbmgr.getResources()),"Machine",resource,24);
               dbmgr.getResources()[resource]=myresource;
               myresource.setAutomated(True);
               myresource.setAvailableShift("All");
               myresource.setOperatingEffort(0.1);
        elif "Outsourced" in resource.split():
            myresource = Resource(len(dbmgr.getResources()),"Outsourced",resource,10000);
            myresource.setAvailableShift("All");
            dbmgr.getResources()[resource]=myresource;
        else:
            myresource = Resource(len(dbmgr.getResources()),"Manual",resource,15);
            if resource == 'MP1 - Material preparation - automatic sawing (ZGN_01)':
                myresource.setOperatingEffort(0.15)
            elif resource == 'MP3 - Material preparation - pre-cutted, only picking (UITG_M)':
                myresource.setOperatingEffort(0)
            else:                
                myresource.setOperatingEffort(1);
            myresource.setAvailableShift("All");
            dbmgr.getResources()[resource]=myresource;
    else:
        myresource = dbmgr.getResources()[resource]; 

    
    newopr = operation

    if myresource !=None:
        myresource.getOperations().append(newopr)
        newopr.getRequiredResources().append(myresource)

    if not newopr.getName() in dbmgr.getOperations():
        dbmgr.getOperations()[newopr.getName()]=newopr;
    else:
        k = 2
        Unique = False
        while Unique is False:
            if not newopr.getName()+"_"+str(k) in dbmgr.getOperations():
                newopr.setName(newopr.getName()+"_"+str(k))
                dbmgr.getOperations()[newopr.getName()]=newopr;
                Unique = True
            else:
                k+=1
        
    return newopr

def SaveInstance(DataMgr,file):

        # Save Products.
        products_df = pd.DataFrame(columns= ["ProductID", "ProductNumber","Name","StockUnit","StockBatch","StockLevel","PrescribedBatchsize","ChosenBatchsize"])
        precedences_df = pd.DataFrame(columns= ["PredecessorID","SuccessorID","Multiplier"])
        prodrops_df = pd.DataFrame(columns= ["ProductID","OperationID","OperationIndex"])
        operations_df = pd.DataFrame(columns= ["OperationID","Name","ProcessTime","Predecessor"])
        opsres_df = pd.DataFrame(columns= ["OperationID","ResourceID"])
        resources_df = pd.DataFrame(columns= ["ResourceID","ResourceType","Name","DailyCapacity","Automated","OperatingEffort","Shift"])
        orders_df = pd.DataFrame(columns= ["OrderID","ProductID","ProductName","Name","Quantity","Deadline"])
 
        
        for name,myprod in DataMgr.Products.items():
            products_df.loc[len(products_df)] = {"ProductID":myprod.getID(), "ProductNumber":myprod.getPN(),"Name":myprod.getName(),"StockUnit":myprod.getStockUnit(),"StockBatch":myprod.getStockBatch(),"StockLevel":myprod.getStockLevel(),"PrescribedBatchsize":myprod.getPrescribedBatchsize(),"ChosenBatchsize":myprod.getChosenBatchsize()}

            
            for pred in myprod.getPredecessors():
                precedences_df.loc[len(precedences_df)] = {"PredecessorID":pred.getID(),"SuccessorID":myprod.getID(),"Multiplier":myprod.getMPredecessors()[pred]}
 
            oprind = 0
            for opr in myprod.getOperations(): 
                prodrops_df.loc[len(prodrops_df)] = {"ProductID":myprod.getID(),"OperationID":opr.getID(),"OperationIndex":oprind}
                oprind+=1 

        for opname,opr in DataMgr.Operations.items():
            pred = ""           
            operations_df.loc[len(operations_df)]= {"OperationID":opr.getID(),"Name":opr.getName(),"ProcessTime":opr.getProcessTime(),"Predecessor":opr.getPredecessor()}
            
            for res in opr.getRequiredResources(): 
                opsres_df.loc[len(opsres_df)]= {"OperationID":opr.getID(),"ResourceID":res.getID()}
        

        for resname,res in DataMgr.Resources.items():
            resources_df.loc[len(resources_df)] ={"ResourceID":res.getID(),"ResourceType":res.getType(),"Name":res.getName(),"DailyCapacity":res.getDailyCapacity(),"Automated":res.getAutomated(),"OperatingEffort":res.getOperatingEffort(),"Shift":res.getAvailableShift()}
 
        
        for ordname,ordr in DataMgr.CustomerOrders.items():
            orders_df.loc[len(orders_df)]={"OrderID":ordr.getID(),"ProductID":ordr.getProduct().getID(),"ProductName":ordr.getProduct().getName(),"Name":ordr.getName(),"Quantity":ordr.getQuantity(),"Deadline":ordr.getDeadLine()}

    


        folder = 'UseCases'; casename = file
        path = folder+"\\"+casename
        isExist = os.path.exists(path)

        if not isExist:
            os.makedirs(path)
       
        
        filename = 'Products.csv'; path = folder+"\\"+casename+"\\"+filename;  fullpath = os.path.join(Path.cwd(), path) 
        products_df.to_csv(path, index=False)
        filename = 'Precedences.csv';path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        precedences_df.to_csv(fullpath, index=False)
        filename = 'ProductsOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        prodrops_df.to_csv(fullpath, index=False)
        filename = 'ResourcesOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        opsres_df.to_csv(fullpath, index=False)
        filename = 'Operations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        operations_df.to_csv(fullpath, index=False)
        filename = 'Resources.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        resources_df.to_csv(fullpath, index=False)
        filename = 'CustomerOrders.csv'; path = folder+"\\"+casename+"\\"+filename;fullpath = os.path.join(Path.cwd(), path)
        orders_df.to_csv(fullpath, index=False)

        return

##Create operators and manual worker
Op1 = Resource(len(DataMgr.getResources()),"Operator","Operator 1",8);
Op1.setAvailableShift(1);
DataMgr.getResources()["Operator 1"]=Op1;

Op2 = Resource(len(DataMgr.getResources()),"Operator","Operator 2",8);
Op2.setAvailableShift(1);
DataMgr.getResources()["Operator 2"]=Op2;

Op3 = Resource(len(DataMgr.getResources()),"Operator","Operator 3",7);
Op3.setAvailableShift(2);
DataMgr.getResources()["Operator 3"]=Op3;

ManWor = Resource(len(DataMgr.getResources()),"Operator","Manual workers",15);
DataMgr.getResources()["Manual workers"]=ManWor;
##End create operators

PrescribedBatches=dict()

for row, rule in stock_df.iterrows():
    PrescribedBatches[rule['Product']]=rule['Multiple Quantity']



nopass = 0;
Stockproducts = list(stock_df['Product'])
for i,r in bom_df.iterrows():
    CreatedOperations=[]
    productrow = False;
    if nopass > 0:
        nopass -=1
        continue

    if isinstance(r['Product'],str):
        if not pd.isnull(r['Product']):
            productrow = True;

    if productrow == True:
        newproduct = MakeProduct(DataMgr,r['Product'],0)
        if r['Product'] in PrescribedBatches:
            newproduct.setPrescribedBatchsize(PrescribedBatches[r['Product']])
        newproduct.setStockUnit(r['BoM Lines/Product Unit of Measure'])
        if r['BoM Lines/Product Unit of Measure'] == 'mm':
            newproduct.setStockBatch(3000)
        DataMgr.getProducts()[newproduct.getName()] = newproduct;

        if not pd.isnull(r['BoM Lines/Component']):
            rawprod=r['BoM Lines/Component']
            if rawprod in Stockproducts:
                stock = stock_df['On Hand'][Stockproducts.index(rawprod)]
            else:
                stock=0

            rawprod = MakeProduct(DataMgr,r['BoM Lines/Component'],stock)
            
            quantity = r['BoM Lines/Quantity']

            newproduct.getMPredecessors()[rawprod] = quantity
            newproduct.getPredecessors().append(rawprod)
            rawprod.setSuccessor(newproduct)
            if r['Product'] in PrescribedBatches:
                rawprod.setPrescribedBatchsize(PrescribedBatches[r['Product']])

            DataMgr.getProducts()[rawprod.getName()] = rawprod

        j = i
        while j < len(bom_df):
            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('Product')]) and j > i:
                break
            #print("Operation: ",bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],"-",bom_df.iloc[j,bom_df.columns.get_loc('Reference')])
            duration = bom_df['BoM Lines/Operations/Duration'][j]
            if isinstance(duration, (int, float)):
                dur = float(1/60)*float(duration) # time assumed to be minute..

            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]):
                if bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')] == "OUT - Outsourced activity":
                    myopr = Operation(len(DataMgr.getOperations()),bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]+"_"+newproduct.getName(),dur)
                    myopr = MakeOperation(DataMgr,myopr,newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')]+"_"+bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],dur)
                    newproduct.getOperations().append(myopr)                    
                    if j > i:
                        myopr.setPredecessor(CreatedOperations[-1])                        
                    CreatedOperations.append(myopr.getName())
                    
                else:
                    myopr = Operation(len(DataMgr.getOperations()),bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]+"_"+newproduct.getName(),dur)
                    myopr = MakeOperation(DataMgr,myopr,newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')],dur)
                    newproduct.getOperations().append(myopr)
                    if j > i:
                        myopr.setPredecessor(CreatedOperations[-1])                        
                    CreatedOperations.append(myopr.getName())
                    
               
   
            j+=1 
        nopass=len(newproduct.getOperations())-1
       
        i = j-1
        #print("i",i,"j",j)
    else:
        print(">>>>>>>>>>>> No Product definition: ", isinstance(r['Product'],str),pd.isnull(r['Product']))
        print("Product value: ",r['Product'])
        print(r)

rel_path = "Sales Order Line (sale.order.line).xlsx"
abs_file_path = os.path.join(Path.cwd(), rel_path)
xls = pd.ExcelFile(abs_file_path)
orders_df = pd.read_excel(xls,"Sheet1")

#orders_df['Delivery Date']  = pd.to_datetime(orders_df['Delivery Date'] )

for i,r in orders_df.iterrows():
    code = r['Description']
    if code.find("[")>-1 and code.find("]")>-1:
        code = code[code.find("[")+1:code.find("]")]
    else:
        continue
    
    ordername = r['Order Reference']+"_("+code+")"

    prodmatch = [prod for prod in DataMgr.getProducts().values() if prod.getPN() == code]

    if len(prodmatch) == 0:
        continue

    myprod = prodmatch[0]
    if myprod.getPrescribedBatchsize() is not None:
        myprod.setChosenBatchsize(myprod.getPrescribedBatchsize())
    else:
        myprod.setChosenBatchsize(r['Quantity']) 

    deadline = datetime.today().strftime('%Y-%m-%d')

    if not pd.isnull(r['Delivery Date']):
        deadline = r['Delivery Date'].strftime('%Y-%m-%d')
    

    #print(r['Delivery Date'],pd.isnull(r['Delivery Date']))
    myorder = CustomerOrder(len(DataMgr.getCustomerOrders()),ordername,myprod.getID()
                                                           ,myprod.getName(),r['Quantity'],str(deadline))
    myorder.setProduct(myprod)

    DataMgr.getCustomerOrders()[ordername] = myorder

print('Nr Products: ',len(DataMgr.getProducts()))
print('Nr Resources: ',len(DataMgr.getResources()))
print('Nr Operations: ',len(DataMgr.getOperations()))
print("Nr Customer Orders: ",len(DataMgr.getCustomerOrders()))

# SaveInstance(DataMgr,'TBRM_Volledige_Instantie')

    

    
    
        

    

Nr Products:  561
Nr Resources:  33
Nr Operations:  1075
Nr Customer Orders:  127


In [1]:
import os
online_version = False;editmode = True
if online_version:  
    user = "muratfirat78";repo = "ProductionPlanning"
    if os.path.isdir(repo):
        !rm -rf {repo}
        
    !git clone https://github.com/{user}/{repo}.git
    %cd /content/{repo}
from DBMain import*
DataMgr.setOnlineVersion(online_version);VisMgr.setEditMode(editmode)
#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); 
tab_5 = VisMgr.getProductionProgressTab().generatePPrgTAB()
tab_3 = VisMgr.generatePLTAB(); tab_6 = VisMgr.getSchedulingTab().generatePSschTAB()
####################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_5,tab_3,tab_6])


tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Instance Data');tab_set.set_title(2, 'Production Progress')
tab_set.set_title(3, 'Planning');tab_set.set_title(4, 'Scheduling')
tab_set

In [2]:
test = DataMgr
